In [157]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
# print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [154]:
# Load the H2O library and start up the H2O cluter locally on your machine
import h2o

# Number of threads, nthreads = -1, means use all cores on your machine
# max_mem_size is the maximum memory (in GB) to allocate to H2O
h2o.init(nthreads = -1, max_mem_size = 8)
from h2o.automl import H2OAutoML

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O cluster uptime:,7 hours 47 mins
H2O cluster timezone:,Europe/Moscow
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.10
H2O cluster version age:,26 days
H2O cluster name:,H2O_from_python_ofitserovvladislav_ama5fh
H2O cluster total nodes:,1
H2O cluster free memory:,5.782 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"


In [150]:
loan_csv = "onetwotrip_challenge_train.csv"  # modify this for your machine
# Alternatively, you can import the data directly from a URL
#loan_csv = "https://raw.githubusercontent.com/h2oai/app-consumer-loan/master/data/loan.csv"
data = h2o.import_file(loan_csv)  # 163,987 rows x 15 columns

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [151]:
data.shape

(196056, 43)

In [167]:
test_loan_csv = "onetwotrip_challenge_test.csv"  # modify this for your machine
# Alternatively, you can import the data directly from a URL
#loan_csv = "https://raw.githubusercontent.com/h2oai/app-consumer-loan/master/data/loan.csv"
test_data = h2o.import_file(test_loan_csv)  # 163,987 rows x 15 columns

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [168]:
test_data.shape

(455011, 37)

In [152]:
splits = data.split_frame(ratios=[0.7, 0.15], seed=1)  

train = splits[0]
valid = splits[1]
test = splits[2]

In [31]:
print(train.nrow)
print(valid.nrow)
print(test.nrow)

137270
29318
29468


In [160]:
x = list(data.columns)

In [161]:
y = 'goal1'

In [162]:
x.remove(y)
x

['orderid',
 'userid',
 'field0',
 'field1',
 'field2',
 'field3',
 'field4',
 'field5',
 'field6',
 'field7',
 'field8',
 'field9',
 'field10',
 'field11',
 'field12',
 'field13',
 'field14',
 'field15',
 'field16',
 'field17',
 'field18',
 'field19',
 'field20',
 'field21',
 'field22',
 'field23',
 'field24',
 'field25',
 'field26',
 'field27',
 'field28',
 'field29',
 'indicator_goal21',
 'indicator_goal22',
 'indicator_goal23',
 'indicator_goal24',
 'indicator_goal25',
 'goal21',
 'goal22',
 'goal23',
 'goal24',
 'goal25']

In [87]:
from h2o.estimators.glm import H2OGeneralizedLinearEstimator

In [88]:
# Initialize the GLM estimator:
# Similar to R's glm() and H2O's R GLM, H2O's GLM has the "family" argument

glm_fit1 = H2OGeneralizedLinearEstimator(family='binomial', model_id='glm_fit1')


In [42]:
glm_fit1.train(x=x, y=y, training_frame=train)

glm Model Build progress: |███████████████████████████████████████████████| 100%


In [97]:
glm_fit2 = H2OGeneralizedLinearEstimator(family='binomial', model_id='glm_fit2', lambda_search=True)
glm_fit2.train(x=x, y=y, training_frame=data)

glm Model Build progress: |███████████████████████████████████████████████| 100%


In [44]:
glm_perf1 = glm_fit1.model_performance(test)
glm_perf2 = glm_fit2.model_performance(test)


In [45]:
# Print model performance
print(glm_perf1)
print(glm_perf2)


ModelMetricsBinomialGLM: glm
** Reported on test data. **

MSE: 0.021840525703319385
RMSE: 0.14778540422964437
LogLoss: 0.10283865457857932
Null degrees of freedom: 29467
Residual degrees of freedom: 29454
Null deviance: 6387.802177228457
Residual deviance: 6060.898946243151
AIC: 6088.898946243151
AUC: 0.6967800274995015
pr_auc: 0.06290692803480724
Gini: 0.39356005499900304

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.058185914705425446: 


,,0,1,Error,Rate
0,0,27941.0,858.0,0.0298,(858.0/28799.0)
1,1,554.0,115.0,0.8281,(554.0/669.0)
2,Total,28495.0,973.0,0.0479,(1412.0/29468.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.058186,0.140073,173.0
1,max f2,0.025247,0.184126,270.0
2,max f0point5,0.062165,0.126933,166.0
3,max accuracy,0.885243,0.977263,0.0
4,max precision,0.168188,0.190476,38.0
5,max recall,0.012085,1.000000,396.0
6,max specificity,0.885243,0.999965,0.0
7,max absolute_mcc,0.058186,0.118460,173.0
8,max min_per_class_accuracy,0.019431,0.638460,317.0
9,max mean_per_class_accuracy,0.018329,0.644440,328.0



Gains/Lift Table: Avg response rate:  2,27 %, avg score:  2,21 %


...

,,0,1,Error,Rate
0,0,27790.0,1009.0,0.035,(1009.0/28799.0)
1,1,545.0,124.0,0.8146,(545.0/669.0)
2,Total,28335.0,1133.0,0.0527,(1554.0/29468.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.056518,0.137625,178.0
1,max f2,0.029469,0.189049,244.0
2,max f0point5,0.120751,0.132170,97.0
3,max accuracy,0.913621,0.977263,0.0
4,max precision,0.221515,0.195652,29.0
5,max recall,0.005748,1.000000,397.0
6,max specificity,0.913621,0.999965,0.0
7,max absolute_mcc,0.053734,0.116577,182.0
8,max min_per_class_accuracy,0.019711,0.638251,300.0
9,max mean_per_class_accuracy,0.021033,0.646466,291.0



Gains/Lift Table: Avg response rate:  2,27 %, avg score:  2,21 %


,,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
0,,1,0.010011,0.125069,6.719161,6.719161,0.152542,0.179450,0.152542,0.179450,0.067265,0.067265,571.916090,571.916090
1,,2,0.020022,0.094611,3.284923,5.002042,0.074576,0.108577,0.113559,0.144014,0.032885,0.100149,228.492311,400.204201
2,,3,0.030033,0.070871,5.375329,5.126471,0.122034,0.082444,0.116384,0.123491,0.053812,0.153961,437.532872,412.647091
3,,4,0.040010,0.054027,3.296096,4.670042,0.074830,0.061641,0.106022,0.108067,0.032885,0.186846,229.609632,367.004162
4,,5,0.050020,0.046172,1.941091,4.123881,0.044068,0.049614,0.093623,0.096369,0.019432,0.206278,94.109093,312.388121
5,,6,0.100007,0.032353,2.182954,3.153747,0.049559,0.037493,0.071598,0.066941,0.109118,0.315396,118.295436,215.374709
6,,7,0.150027,0.027564,1.494160,2.600426,0.033921,0.029605,0.059036,0.054493,0.074738,0.390135,49.415986,160.042622
7,,8,0.200014,0.024819,1.375560,2.294314,0.031229,0.026130,0.052087,0.047404,0.068759,0.458894,37.556028,129.431364
8,,9,0.300020,0.021276,1.240574,1.943067,0.028164,0.022862,0.044113,0.039224,0.124066,0.582960,24.057350,94.306693
9,,10,0.399993,0.018991,0.912056,1.685380,0.020706,0.020055,0.038262,0.034433,0.091181,0.674141,-8.794372,68.537987


In [46]:
# Retreive test set AUC
print(glm_perf1.auc())
print(glm_perf2.auc())


0.6967800274995015
0.701080905535096


In [47]:
# Compare test AUC to the training AUC and validation AUC
print(glm_fit2.auc(train=True))
print(glm_fit2.auc(valid=True))

0.703675585561129
0.7016205068564219


In [48]:
# Import H2O RF:
from h2o.estimators.random_forest import H2ORandomForestEstimator

In [49]:
# Initialize the RF estimator:

rf_fit1 = H2ORandomForestEstimator(model_id='rf_fit1', seed=1)

In [50]:
rf_fit1.train(x=x, y=y, training_frame=train)


drf Model Build progress: |███████████████████████████████████████████████| 100%


In [51]:
rf_fit2 = H2ORandomForestEstimator(model_id='rf_fit2', ntrees=100, seed=1)
rf_fit2.train(x=x, y=y, training_frame=train)

drf Model Build progress: |███████████████████████████████████████████████| 100%


In [52]:
rf_perf1 = rf_fit1.model_performance(test)
rf_perf2 = rf_fit2.model_performance(test)

In [57]:
print(rf_perf1)
print(rf_perf2)


ModelMetricsRegression: drf
** Reported on test data. **

MSE: 0.022109229080888818
RMSE: 0.14869172499130145
MAE: 0.04570569551175729
RMSLE: 0.10479392726390226
Mean Residual Deviance: 0.022109229080888818


ModelMetricsRegression: drf
** Reported on test data. **

MSE: 0.021944306459815426
RMSE: 0.1481361078866845
MAE: 0.04572805230421732
RMSLE: 0.10417005972891896
Mean Residual Deviance: 0.021944306459815426



In [58]:
rf_fit3 = H2ORandomForestEstimator(model_id='rf_fit3', seed=1, nfolds=5)
rf_fit3.train(x=x, y=y, training_frame=data)

drf Model Build progress: |███████████████████████████████████████████████| 100%


In [61]:
print(rf_fit3)

Model Details
H2ORandomForestEstimator :  Distributed Random Forest
Model Key:  rf_fit3


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,50.0,50.0,1990838.0,20.0,20.0,20.0,2442.0,3659.0,3138.04




ModelMetricsRegression: drf
** Reported on train data. **

MSE: 0.021972172303533658
RMSE: 0.1482301329134318
MAE: 0.0449173220638826
RMSLE: 0.1052251023591801
Mean Residual Deviance: 0.021972172303533658

ModelMetricsRegression: drf
** Reported on cross-validation data. **

MSE: 0.021541033204375587
RMSE: 0.14676863835430098
MAE: 0.045196667268698226
RMSLE: 0.10355885818809704
Mean Residual Deviance: 0.021541033204375587

Cross-Validation Metrics Summary: 


,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,mae,0.04519763,4.3660146E-4,0.044543035,0.04568315,0.045073234,0.0454853,0.045203418
1,mean_residual_deviance,0.0215425,6.8907614E-4,0.02051618,0.022351006,0.021323789,0.021917567,0.021603962
2,mse,0.0215425,6.8907614E-4,0.02051618,0.022351006,0.021323789,0.021917567,0.021603962
3,r2,0.005129179,0.003586215,0.008492299,0.0031168442,0.008689829,2.918276E-4,0.005055096
4,residual_deviance,0.0215425,6.8907614E-4,0.02051618,0.022351006,0.021323789,0.021917567,0.021603962
5,rmse,0.14675851,0.0023553122,0.1432347,0.14950253,0.14602667,0.14804582,0.14698286
6,rmsle,0.1035529,0.0015676931,0.101234935,0.10531859,0.10300462,0.1045689,0.10363744



Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance
0,,2019-12-04 02:35:22,4 min 59.050 sec,0.0,NaN,NaN,NaN
1,,2019-12-04 02:35:23,4 min 59.813 sec,1.0,0.184081,0.044792,0.033886
2,,2019-12-04 02:35:24,5 min 0.634 sec,2.0,0.181772,0.044711,0.033041
3,,2019-12-04 02:35:25,5 min 1.524 sec,3.0,0.178803,0.044808,0.031971
4,,2019-12-04 02:35:26,5 min 2.428 sec,4.0,0.177007,0.045159,0.031331
5,,2019-12-04 02:35:30,5 min 6.834 sec,7.0,0.168289,0.045153,0.028321
6,,2019-12-04 02:35:35,5 min 11.508 sec,13.0,0.157595,0.045040,0.024836
7,,2019-12-04 02:35:39,5 min 16.361 sec,18.0,0.153840,0.045018,0.023667
8,,2019-12-04 02:35:44,5 min 21.076 sec,24.0,0.151610,0.045044,0.022986
9,,2019-12-04 02:35:49,5 min 25.977 sec,28.0,0.150671,0.045044,0.022702



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,userid,8727.914062,1.000000,0.109148
1,orderid,5192.438965,0.594923,0.064935
2,field16,4881.517578,0.559299,0.061047
3,field11,4607.366211,0.527889,0.057618
4,field23,4561.315918,0.522612,0.057042
5,field14,3503.442383,0.401407,0.043813
6,field18,3120.925293,0.357580,0.039029
7,field1,3091.162842,0.354170,0.038657
8,field12,3078.129150,0.352676,0.038494
9,field20,2918.227051,0.334356,0.036494



See the whole table with table.as_data_frame()



In [62]:
# Import H2O GBM:
from h2o.estimators.gbm import H2OGradientBoostingEstimator

In [63]:
# Initialize and train the GBM estimator:

gbm_fit1 = H2OGradientBoostingEstimator(model_id='gbm_fit1', seed=1)
gbm_fit1.train(x=x, y=y, training_frame=train)

gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [64]:
gbm_fit2 = H2OGradientBoostingEstimator(model_id='gbm_fit2', ntrees=500, seed=1)
gbm_fit2.train(x=x, y=y, training_frame=train)

gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [66]:
# Now let's use early stopping to find optimal ntrees

gbm_fit3 = H2OGradientBoostingEstimator(model_id='gbm_fit3', 
                                        ntrees=500, 
                                        score_tree_interval=5,     #used for early stopping
                                        stopping_rounds=3,         #used for early stopping
                                        stopping_metric='AUC',     #used for early stopping
                                        stopping_tolerance=0.0005, #used for early stopping
                                        seed=1)

# The use of a validation_frame is recommended with using early stopping
gbm_fit3.train(x=x, y=y, training_frame=train, validation_frame=valid)

H2OResponseError: ModelBuilderErrorV3  (water.exceptions.H2OModelBuilderIllegalArgumentException):
    timestamp = 1575416426182
    error_url = '/3/ModelBuilders/gbm'
    msg = 'Illegal argument(s) for GBM model: gbm_fit3.  Details: ERRR on field: _stopping_metric: Stopping metric cannot be AUC for regression.'
    dev_msg = 'Illegal argument(s) for GBM model: gbm_fit3.  Details: ERRR on field: _stopping_metric: Stopping metric cannot be AUC for regression.'
    http_status = 412
    values = {'messages': [{'_log_level': 5, '_field_name': '_keep_cross_validation_models', '_message': 'Only for cross-validation.'}, {'_log_level': 5, '_field_name': '_keep_cross_validation_predictions', '_message': 'Only for cross-validation.'}, {'_log_level': 5, '_field_name': '_keep_cross_validation_fold_assignment', '_message': 'Only for cross-validation.'}, {'_log_level': 5, '_field_name': '_fold_assignment', '_message': 'Only for cross-validation.'}, {'_log_level': 5, '_field_name': '_tweedie_power',

In [67]:
gbm_perf1 = gbm_fit1.model_performance(test)
gbm_perf2 = gbm_fit2.model_performance(test)
# gbm_perf3 = gbm_fit3.model_performance(test)

In [69]:
# Retreive test set AUC
print(gbm_perf1)
print(gbm_perf2)
# print(gbm_perf3.auc())


ModelMetricsRegression: gbm
** Reported on test data. **

MSE: 0.021717872350683433
RMSE: 0.14736984885207502
MAE: 0.042801036645432534
RMSLE: 0.1024833028733818
Mean Residual Deviance: 0.021717872350683433


ModelMetricsRegression: gbm
** Reported on test data. **

MSE: 0.021770524223821378
RMSE: 0.14754837926531547
MAE: 0.042807142138846496
RMSLE: 0.10280643982569465
Mean Residual Deviance: 0.021770524223821378



## --

In [164]:
aml = H2OAutoML(max_models=30, seed=45, max_runtime_secs=3600)
aml.train(x=x, y=y, training_frame=data)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [165]:
# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
XGBoost_1_AutoML_20191204_144448,0.0211493,0.145428,0.0211493,0.0429174,0.101372
XGBoost_1_AutoML_20191204_155733,0.0211493,0.145428,0.0211493,0.0429174,0.101372
XGBoost_2_AutoML_20191204_155733,0.0214094,0.146319,0.0214094,0.0442916,0.102721
XGBoost_3_AutoML_20191204_155733,0.0477517,0.218522,0.0477517,0.191095,0.183156


In [166]:
# The leader model is stored here
aml.leader

Model Details
H2OXGBoostEstimator :  XGBoost
Model Key:  XGBoost_1_AutoML_20191204_144448


Model Summary: 


,,number_of_trees
0,,122.0




ModelMetricsRegression: xgboost
** Reported on train data. **

MSE: 0.019263920485397514
RMSE: 0.1387945261362908
MAE: 0.04097542225020642
RMSLE: 0.09572492384177299
Mean Residual Deviance: 0.019263920485397514

ModelMetricsRegression: xgboost
** Reported on cross-validation data. **

MSE: 0.02114934223589959
RMSE: 0.1454281342653463
MAE: 0.04291740046825409
RMSLE: 0.10137161092697416
Mean Residual Deviance: 0.02114934223589959

Cross-Validation Metrics Summary: 


,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,mae,0.042917404,3.957213E-4,0.042479288,0.043451786,0.04318904,0.0426726,0.0427943
1,mean_residual_deviance,0.021149345,5.499313E-4,0.020588141,0.021869088,0.02158108,0.020764451,0.020943966
2,mse,0.021149345,5.499313E-4,0.020588141,0.021869088,0.02158108,0.020764451,0.020943966
3,r2,0.023166597,0.0035217206,0.022439841,0.020578472,0.027113494,0.026497902,0.019203275
4,residual_deviance,0.021149345,5.499313E-4,0.020588141,0.021869088,0.02158108,0.020764451,0.020943966
5,rmse,0.14541835,0.0018870307,0.14348568,0.147882,0.146905,0.14409876,0.1447203
6,rmsle,0.101365305,0.001264994,0.10011025,0.10307154,0.10228276,0.100420095,0.10094187



Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance
0,,2019-12-04 15:24:42,39 min 52.403 sec,0.0,0.500000,0.500000,0.250000
1,,2019-12-04 15:24:53,40 min 2.891 sec,5.0,0.397521,0.396317,0.158023
2,,2019-12-04 15:25:02,40 min 12.012 sec,10.0,0.320556,0.315915,0.102756
3,,2019-12-04 15:25:10,40 min 20.567 sec,15.0,0.263872,0.253726,0.069628
4,,2019-12-04 15:25:20,40 min 29.737 sec,20.0,0.222919,0.205478,0.049693
5,,2019-12-04 15:25:29,40 min 39.057 sec,25.0,0.194144,0.168153,0.037692
6,,2019-12-04 15:25:37,40 min 47.405 sec,30.0,0.174556,0.139219,0.030470
7,,2019-12-04 15:25:46,40 min 55.735 sec,35.0,0.161709,0.116870,0.026150
8,,2019-12-04 15:25:53,41 min 3.361 sec,40.0,0.153408,0.099580,0.023534
9,,2019-12-04 15:26:03,41 min 12.877 sec,45.0,0.148101,0.086159,0.021934



See the whole table with table.as_data_frame()

Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,field16,422.965485,1.000000,0.096852
1,goal24,271.233002,0.641265,0.062108
2,orderid,256.995422,0.607604,0.058848
3,field12,218.551941,0.516713,0.050045
4,field11,205.851532,0.486686,0.047137
5,field6,201.524734,0.476457,0.046146
6,field23,195.452606,0.462101,0.044756
7,field13,189.884125,0.448935,0.043480
8,field1,187.382172,0.443020,0.042908
9,field14,174.978745,0.413695,0.040067



See the whole table with table.as_data_frame()


In [147]:
predictions = gbm_fit2.predict(test_data)

gbm prediction progress: |████████████████████████████████████████████████| 100%


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

/Users/ofitserovvladislav/anaconda3/lib/python3.7/site-packages/h2o/job.py:70: UserWarning: Test/Validation dataset is missing column 'goal21': substituting in a column of NaN
  warnings.warn(w)
/Users/ofitserovvladislav/anaconda3/lib/python3.7/site-packages/h2o/job.py:70: UserWarning: Test/Validation dataset is missing column 'goal22': substituting in a column of NaN
  warnings.warn(w)
/Users/ofitserovvladislav/anaconda3/lib/python3.7/site-packages/h2o/job.py:70: UserWarning: Test/Validation dataset is missing column 'goal23': substituting in a column of NaN
  warnings.warn(w)
/Users/ofitserovvladislav/anaconda3/lib/python3.7/site-packages/h2o/job.py:70: UserWarn

In [148]:
predictions

predict
0.00876425
0.0034028
0.0246613
0.0228645
0.000393138
0.0110437
0.0210835
0.00868058
0.0128389
0.0133872


In [139]:
predictions[:,[2]]

p1
0.00940842
0.0131095
0.0199982
0.0262801
0.0118402
0.0128222
0.0173326
0.0153197
0.0183538
0.0161764


In [141]:
pred = predictions[:,2]
# summary(pred)
submission = data.frame(Unnamed=test, Proba=pred)

# cat("saving the submission file\n")
# write.csv(submission, "h2o_rf.csv")

AttributeError: 'H2OFrame' object has no attribute 'frame'

In [171]:
sample_submission = pd.read_csv('sub1.csv')
sample_submission

,orderid,proba
0,0,0.008961
1,1,0.007810
2,2,0.032878
3,3,0.028098
4,4,0.004727
...,...,...
455006,455006,0.074678
455007,455007,0.024433
455008,455008,0.013714
455009,455009,0.043056


In [149]:
sample_submission['proba'] = predictions.as_data_frame().values
sample_submission.to_csv('h2o_gbm.csv', index=False)

In [169]:
preds = aml.predict(test)

xgboost prediction progress: |████████████████████████████████████████████| 100%


In [170]:
 preds

predict
0.0162555
0.0163959
0.0132639
0.018784
0.0083389
0.0539366
0.0102523
0.016237
0.0485733
0.00998038


In [174]:
sample_submission['proba'] = preds.as_data_frame()
sample_submission.to_csv('autoML.csv', index=False)